# Homework 3

We first import the needed libraries and set the elements we need for the rest of the Homework.

In [ ]:
# Used to look for the saved data sheets
DATA_FOLDER = 'Data/'
IMAGE_FOLDER = 'Data/Images/'

In [ ]:
# Importing libraries
import os
import json
import seaborn as sns #might not be useful
import folium
from IPython.display import Image #permits to show images

In [ ]:
#Verifying we have the right version
folium.__version__ == '0.5.0'

## Unemployment in Europe

The very first thing we have to think about is what data we want to use. Eurostat has a lot of types of information, even more so, has information per country, but also per province/state of some countries (known as NUTS 2 regions). Thus, we have to determine what kind of information - and how much - we want to show.

Thus, we decide to take the **yearly** average for the unemployment rates and not the monthly ones, thus getting data from 2016 and not from 2017. For the monthly rates, we could have information for the beginning of this year, but we choose against it. This also permits to not have any fluctuation due to seasonal workers that would not be seen as unemployed for a time.

Secondly, seeing as the idea is to compare the unemployment rate of Europe to that of Switzerland, it would be more appropriate to take the data by country rather than that per NUTS 2 region, even more so seeing as Switzerland is cut in several in that dataset. So, to be able to compare it to Switzerland as a whole, we should have Switzerland as only one entity.

In [ ]:
Image(IMAGE_FOLDER + 'euro01.png')

We also had the option to download the data in several different types of files, like HTML, CSV, TSV or XLS. We choose to download the data in a XLS format.

In [ ]:
Image(IMAGE_FOLDER + 'euro02.png')

## Unemployment in Switzerland

Here, we need several things to be contained in our data, as this is a question that contains several points. First, we need to see each canton individually, but also have some statistics of people who alreay have a job but search for another one. 

First, on amstat, we go to the category "Chômeurs" and then look at the details.

In [ ]:
Image(IMAGE_FOLDER + 'amstat01.png')

Then, we choose "Taux de chômage".

In [ ]:
Image(IMAGE_FOLDER + 'amstat02.png')

 We also choose to look at the whole year in review, which means all of 2017, so that we can maybe do a few comparaisons. Thus we choose "À partir du janvier de l'année sous revue". 

We also select three indicators: "Chômeurs inscrits", "Chômeurs jeunes" and "Chômeurs de longue durée" to see all the types of unemployed people.

In [ ]:
Image(IMAGE_FOLDER + 'amstat03.png')

We take both "Demandeur d'emploi" and "Demandeur d'emploi non chômeur" as to know who is or isn't really unemployed.

Next, for the geography part, we select all three (Language, region and canton), mainly so that we can maybe use this dataset later for the Röstigraben question.

In [ ]:
Image(IMAGE_FOLDER + 'amstat04.png')

Lastly, we also select the option "Classes d'âge 15-24, 25-49, 50 ans et plus" as we will be able to use it to see the different categories of unemplyed people and use it in the last part of the question.

In [ ]:
Image(IMAGE_FOLDER + 'amstat05.png')

Now we can select what type of file we want the file in. We chose the XLS file as "Excel avec texte clair" as the other XLS files didn't seem appropriate to us, because of a few problems with the colapsing of rows (for the regions).

In [ ]:
Image(IMAGE_FOLDER + 'amstat07.png')

### Differences in unemployment between foreigners and natives

Here we have two different questions again: one about the difference in employment regarding nationality, and the second regarding age. Seeing as we already have the information regarding age in the first dataset, we do not need to find another one. Regarding nationality, we proceed similarly as above.

Thus, we choose exactly the same information as before but at the last option, we select "Nationalité" instead of the age classes.

In [ ]:
Image(IMAGE_FOLDER + 'amstat06.png')

### The Röstigraben